# Set Packing問題
この問題は[Exact Cover](tutorial007_exact_cover.ipynb)問題と似ています。

ある自然数の集合Uを考えます。またその自然数を含むいくつかのグループ$V_{1}, V_{2}, \ldots, V_{N}$を想定します。１つの自然数が複数のグループに属していても構いません。さて、そのグループ$V_{i}$からいくつかピックアップしたときに、それらに同じ自然数が複数回含まれず、Uに含まれる自然数セットと同じになるようにピックアップする問題がExact Cover問題でした。このとき、選んだグループ数が最大になるように選ぶ問題をSet Packing問題といいます。

## 準備
これをwildqatを使用して解いてみます。
wildqatがインストールされていない場合は、環境に併せて以下のようにインストールしてください。
```bash
pip install wildqat
```

必要なライブラリをimportし、wildqatオブジェクトをインスタンス化します。

In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import wildqat as wq

## QUBOの作成
解きたい問題のコスト関数は

$ E = E_{A} + E_{B} $

で、$E_{A}, E_{B}$はそれぞれ、

$ E _ { A } = A \sum _ { i , j : V _ { i } \cap V _ { j } \neq \emptyset } x _ { i } x _ { j }$

$E _ { B } = - B \sum _ { i } x _ { i }$

と定義されています。

まず、$E_{A}$は、$i \not= j$である$V_{i}, V_{j}$を選んだときに、重複する自然数があったばあいにペナルティを与えることを意味しています。また、$E_{B}$は最も多く選択されている場合に、コストが低くなることを意味しています。

係数$A, B$については、1つ多く選ぶ（コストがB下がる）より、重複がない（1つ重複があるとコストがA上がる）ことを優先したいので、$A > B$である必要があります。

これをコードにすると次の通りです。

In [2]:
A = 1.0
B = 0.9

def get_qubo(V):
    Q = np.zeros( (len(V), len(V)) )

    for i in range(len(V)):
        for j in range(i, len(V)):
            if i == j:
                Q[i][j] += -B
            elif len(V[i]) + len(V[j]) != len( set(V[i]) | set(V[j]) ):
                Q[i][j] +=  A

    return Q

また、結果を表示する関数も定義しておきましょう。

In [3]:
def show_answer(list_x, energies = None, show_graph = False):
    print("Result x:", list_x)
    text = ""
    for i in range(len(list_x)):
        if(list_x[i]):
            text += str(V[i])
    print("Picked {} group(s): {}".format(sum(list_x), text))
    if energies is not None:
        print("Energy:", a.E[-1])
    if show_graph:
        plt.plot(a.E)
        plt.show()

次の通り実行してみます

In [10]:
V = [ [1,2], [3,4,5,6], [7,8,9,10], [1,3,5], [10], [7,9], [2,4,6,8], [1,2,3,4,5,6,8,10] ]
for i in range(5):
    print("---{}回目".format(i+1))
    a = wq.opt()
    a.qubo = get_qubo(V)
    answer = a.sa()
    show_answer(answer, a.E)

---1回目
1.804011583328247
Result x: [0, 0, 0, 1, 1, 1, 1, 0]
Picked 4 group(s): [1, 3, 5][10][7, 9][2, 4, 6, 8]
Energy: -3.6000000000000005
---2回目
1.720658779144287
Result x: [1, 1, 0, 0, 1, 1, 0, 0]
Picked 4 group(s): [1, 2][3, 4, 5, 6][10][7, 9]
Energy: -3.5999999999999996
---3回目
1.751054048538208
Result x: [0, 0, 0, 1, 1, 1, 1, 0]
Picked 4 group(s): [1, 3, 5][10][7, 9][2, 4, 6, 8]
Energy: -3.6000000000000005
---4回目
2.076296091079712
Result x: [0, 0, 0, 1, 1, 1, 1, 0]
Picked 4 group(s): [1, 3, 5][10][7, 9][2, 4, 6, 8]
Energy: -3.6000000000000005
---5回目
2.0937278270721436
Result x: [1, 1, 0, 0, 1, 1, 0, 0]
Picked 4 group(s): [1, 2][3, 4, 5, 6][10][7, 9]
Energy: -3.5999999999999996


正しい答えは、{1,3,5},{10},{7,9},{2,4,6,8}ですが、時々ややコストが高い間違った答えが選ばれてしまいます。

## Maximal Independent Set (MIS)問題との相似性
なお、この問題は、グラフ理論におけるMIS問題と同じ問題になります。
MIS問題とは、無向グラフの頂点を線で結んだうえで、線で結ばれた2つの頂点の両方に色を塗らないようにしつつ、できるだけたくさんの頂点に色を塗る問題です。